# فاز 1

## بارگذاری داده ها

In [4]:
!pip install kagglehub[pandas-datasets]


In [21]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd


# Set the path to the file you'd like to load
file_path = "ecommerceDataset.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "saurabhshahane/ecommerce-text-classification",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

/tmp/ipython-input-21-4224149474.py:12: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:    Household  \
0  Household   
1  Household   
2  Household   
3  Household   
4  Household   

  Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour a

In [22]:
import pandas as pd

# مسیر فایل
file_path = "/kaggle/input/ecommerce-text-classification/ecommerceDataset.csv"

# بارگذاری داده‌ها با تعیین نام ستون‌ها
df = pd.read_csv(file_path, header=None, names=["category", "text"])

# نمایش ۵ نمونه اول
df.head()


,category,text
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


## تابع پیش‌پردازش متن

In [33]:
import re
import string
import nltk

nltk.download("stopwords")   # کلمات توقف (Stopwords)
nltk.download("punkt")       # برای توکن‌سازی متن
nltk.download("wordnet")     # برای Lemmatization

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
def preprocess(text):
    # اطمینان از اینکه ورودی رشته است
    if not isinstance(text, str):
        return ""

    stop_words = set(stopwords.words("english"))  # ساختن مجموعه کلمات توقف
    lemmatizer = WordNetLemmatizer()              # ایجاد آبجکت برای مثلا تبدیل فعل ها به ریشه

    # lowercase
    text = text.lower()

    # حذف URLها
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)

    # حذف ایمیل‌ها
    text = re.sub(r'\S+@\S+', '', text)

    # حذف اعداد تنها یا ترکیب‌شده
    text = re.sub(r'\b\d+\b', '', text)

    # حذف HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # حذف punctuation
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)

    # حذف کاراکترهای تکراری کش‌دار (مثلاً soooo → so)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # حذف فضای اضافه بین کلمات
    text = re.sub(r'\s+', ' ', text).strip()

    # split (توکن‌سازی ساده)
    tokens = text.split()

    # حذف stopwords و Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 2]

    return ' '.join(tokens)



In [41]:
# اعمال پیش‌پردازش
df["clean_text"] = df["text"].apply(preprocess)

# نمایش نمونه‌ها
df[["text", "clean_text"]].head()


,text,clean_text
0,Paper Plane Design Framed Wall Hanging Motivat...,paper plane design framed wall hanging motivat...
1,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",saf floral framed painting wood inch inch spec...
2,SAF 'UV Textured Modern Art Print Framed' Pain...,saf textured modern art print framed painting ...
3,"SAF Flower Print Framed Painting (Synthetic, 1...",saf flower print framed painting synthetic inc...
4,Incredible Gifts India Wooden Happy Birthday U...,incredible gift india wooden happy birthday un...


# فاز 2

##TF-IDF

In [37]:
import numpy as np
from collections import Counter, defaultdict
import math

# متون تمیز شده
documents = df["clean_text"].tolist()
N = len(documents)  # تعداد کل اسناد

# 1. محاسبه DF برای هر واژه
df_counts = defaultdict(int)  # document frequency

for doc in documents:
    unique_words = set(doc.split())
    for word in unique_words:
        df_counts[word] += 1

# 2. محاسبه IDF برای هر واژه
idf = {}
for word, df_count in df_counts.items():
    idf[word] = math.log(N / (1 + df_count))  # افزودن 1 برای جلوگیری از تقسیم بر صفر


In [38]:
# تمام کلمات موجود (واژگان نهایی)
vocab = sorted(idf.keys())
word2idx = {word: idx for idx, word in enumerate(vocab)}

# ایجاد ماتریس ویژگی N × V
tfidf_matrix = np.zeros((N, len(vocab)))

for i, doc in enumerate(documents):
    word_counts = Counter(doc.split())
    total_words = sum(word_counts.values())

    for word, count in word_counts.items():
        if word in idf:
            tf = count / total_words
            tfidf_matrix[i, word2idx[word]] = tf * idf[word]


In [39]:
#print the first one
print(documents[0])  # سند پیش‌پردازش شده
for word in documents[0].split():
    if word in idf:
        print(word, "→ tf-idf:", idf[word])
    else:
        print(word, "❌ not in vocab")



paper plane design framed wall hanging motivational office decor art print inch set painting made synthetic frame textured print give multi effect attracts towards special series painting make wall beautiful give royal touch painting ready hang would proud possess unique painting niche apart use modern efficient printing technology print ink precision epson roland printer innovative printing technique result durable spectacular looking print highest last lifetime print solely topnotch ink achieve brilliant true colour due high level resistance print retain beautiful colour many year add colour style living space digitally printed painting pleasure eternal blisso bring home elegant print lushed rich color make nothing sheer elegance friend familyit would treasured forever whoever lucky recipient liven place intriguing painting high definition graphic digital print home office room
paper → tf-idf: 3.4130474215796514
plane → tf-idf: 6.032431985596205
design → tf-idf: 1.8288501346008397
fr